##### Library

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import re
from konlpy.tag import Okt, Kkma, Hannanum
from eunjeon import Mecab

from tqdm.notebook import tqdm

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, f1_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import lightgbm

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

__STEP 00 :__ Data Load 및 전처리

In [2]:
#finalDF = pd.read_csv('useModelingData_label2.csv')
finalDF = pd.read_csv('data/useModelingData_label2_threshold3.csv')


#년 월 데이터만 활용하기로
finalDF['시각'] = finalDF['시각'].str[:7].apply(lambda x : int(x.replace('.', '')))

# 감정표현 string => list 변환
finalDF['감정표현'] = finalDF['감정표현'].apply(lambda x : [int(i) for i in x[1:-1].split(', ')])

finalDF['좋아요'] = finalDF['감정표현'].apply(lambda x : x[0])
finalDF['훈훈해요'] = finalDF['감정표현'].apply(lambda x : x[1])
finalDF['슬퍼요'] = finalDF['감정표현'].apply(lambda x : x[2])
finalDF['화나요'] = finalDF['감정표현'].apply(lambda x : x[3])
finalDF['후속기사원해요'] = finalDF['감정표현'].apply(lambda x : x[4])

finalDF.tail(2)

,시각,카테고리,미디어,제목,내용,수정시간,감정표현,label,화나요rate,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
38276,202012,사회,조선일보,野동의 없는 ‘반쪽 처장’ 논란에… 金 “정치중립이 최우선”,초대 공수처장 지명 김진욱 공수처장 내정자는 김진욱 54 고위공직자범죄수사처 공수처...,0,"[2, 0, 1, 52, 1]",1,0.928571,2,0,1,52,1
38277,202012,사회,서울신문,눈물 쏟은 이재용 “너무나 존경하는 아버지께 효도를…” 최후진술종합,이재용 국정농단 사건 파기환송심 결심공판 최후진술 최후진술서 부친 이건희 회장 별세...,1,"[12, 2, 509, 333, 7]",1,0.385863,12,2,509,333,7


__STEP 01 :__ 형태소 분석기 활용 및 정제
- Mecab 형태소 분석기 활용 (빠른 속도)

In [3]:
tokenizer = Mecab()
#tokenizer = Okt()

#제거하고자 하는 특수문자
string_punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~…’'
re_tok = re.compile(f'([{string_punctuation}])')



# 자주 등장하며, 불용어가 아닌 1글자 단어 List
useLen1WordList = [
                '년','월','주','문','군','적','핵','명','조','억','또','법','측','뒤','힘','앉',
                '글','열','신','률','왔','찾','놓','못','않','없','때','많','딸','뜻','왜','끝',
                '곧','액','꿈','뽑','촌','잃','밖','믿','탈','피','편','쉽','첫','본','박','꾼',
                '받','앞','속,','중','꽃','집','답','감','개','날','대','분','것','째','전','안','돼',
                   
                '文','與','野','美','日','韓','中','靑','北','軍','無','前',
                '檢','反','法','黃','秋','發','康','南','朴','男','女','尹','故']
 



finalDF['제목'] = finalDF['제목'].apply(lambda title : re_tok.sub(r'', title))
finalDF['내용'] = finalDF['내용'].apply(lambda content : re_tok.sub(r'', content))


token_title_list = []
token_content_list = []


for text in tqdm(finalDF['제목']):
    token = tokenizer.morphs(text)
    token = [t for t in token if (len(t) >= 2) | (t in useLen1WordList)]
    token_title_list.append(token)

    
for text in tqdm(finalDF['내용']):
    text = re_tok.sub(r'', text)
    token = tokenizer.morphs(text)
    token = [t for t in token if (len(t) >= 2) | (t in useLen1WordList)]
    token_content_list.append(token)

In [4]:
print(finalDF['제목'][10028])
print(token_title_list[10028])

집값 떨어지는데 금리마저 들썩 가계부채 뇌관 터지나
['집값', '떨어지', '는데', '금리', '마저', '들썩', '가계', '부채', '뇌관', '터지']


In [5]:
token_title_list[:20]

[['박순자', '국토', '위원장', '고수', '완강', '국당', '윤리', '회부', '논의'],
 ['국민', '힘', '이낙연', '진정', '인정', '려면', '라임', '옵티머스', '특검', '수용', '해야'],
 ['韓', 'IMD', '국가', '경쟁력', '28', '23', '상승', '인구', '천만', '국가', '중', '종합'],
 ['조국', '기자', '뻗치', '취재', '저격', '언론', '자유', '한계', '없'],
 ['이낙연', '검찰', '개혁', '지속', '해야', '추미애', '초석'],
 ['주택',
  '연금',
  '분야',
  '전세',
  '오피스텔',
  '가입',
  '가능',
  '임대',
  '연금',
  '소득',
  '으로',
  '노후',
  '보장',
  '인구',
  '변화',
  '대책'],
 ['동네', '식당', '음식', '던지', '행패', '112', '허위', '신고', '50', '대', '구속'],
 ['22', '부터', '유럽', '입국', '코로나', '19', '검사', '주', '격리', '무화', '종합'],
 ['부산시', '국감', '유재수', '경제', '부시장', '감찰', '내용', '집중', '추궁'],
 ['文', '대통령', '미래', '청사진', '발표', '2030', '년', '세계'],
 ['이해찬', '국민', '성원', '감사', '앞', '으로', '한반도', '평화', '위해', '노력'],
 ['초점', '민주당', '맡겼', '텃밭', '순천', '10', '년', '탈환'],
 ['미래', '통합', '합류', '박결', '전', '대표'],
 ['화순', '병원', '철인', '선수단', '500', '훈련', '용품', '지원'],
 ['코로나',
  '19',
  '일확',
  '진자',
  '144',
  '284',
  '505',
  '명',
  '중국',
  '뛰어넘',
  '1766',
  '명',
  '

__STEP 02 :__ doc2vec 을 활용한 제목+내용 Embedding

In [6]:
#제목 + 내용
total_token = [title + content for title, content in zip(token_title_list, token_content_list)]


# Tagging for each document
# doc2vec 모듈 특성상, 각 뉴스별로 Tag 를 설정해줘야함
# 모든 기사에 대해 각 기사별로 tag값을 다르게 설정 ex) news_tag1 ~ news_tag38277
tagged_data  = [TaggedDocument(d, ['news_tag{}'.format(i)]) for i, d in enumerate(total_token)]


# doc2vec model parameter settings
doc2vec_params = {
    'documents' : tagged_data, 
    'vector_size' : 300,
    'window' : 7,
    'min_count' : 2,
    'alpha' : 0.0025
    }


doc2vec_model = Doc2Vec(**doc2vec_params)

In [7]:
X_text = doc2vec_model.docvecs.vectors_docs  # X_text size : 300

__STEP 03 :__ 카테고리 Label Encoding
- 정치 : 0
- 경제 : 1
- 사회 : 2

In [8]:
finalDF['카테고리'] = finalDF['카테고리'].apply(lambda x : 0 if x == '정치' else 1 if x == '경제' else 2)

__STEP 04 :__ input X dataset 만들기

In [9]:
use_col_list = ['시각', '수정시간', '화나요rate', '카테고리',
                '좋아요', '훈훈해요', '슬퍼요', '화나요', '후속기사원해요']


X_values = finalDF[use_col_list]


X = np.hstack((X_text, X_values))   # X_text : 300 dim  / X_values dim : 9
X = pd.DataFrame(X) # Modeling 후에 , test data를 통한 검증을 할 때 , index값을 뽑아내기 위해 형 변환


y = finalDF['label'].values


print('X shape : {}'.format(X.shape))
print('y shape : {}'.format(y.shape))

X shape : (38278, 309)
y shape : (38278,)


## <center>Modeling</center>

__STEP 05 :__ model Scoring
- Accuracy(정확도)

In [10]:
def model_scoring(model, X_test, y_test):
    predict = model.predict(X_test)
    print('\n\n#####classification_report#####')
    print(classification_report(y_test, predict))
    print('\n\n#####confusion Matrix#####')
    print(confusion_matrix(y_test, predict))
    display(pd.DataFrame(confusion_matrix(y_test, predict)))
    
    print('\n\n정확도 : ', accuracy_score(y_test, predict))

__STEP 06 :__ KFold LightGBM 
- 8 - Fold 

In [11]:
def printKfoldAcc(model, k = 10):
    ## default parameter "K" = 10


    print ('#### model : {} ####'.format(model))
    kfold  = KFold(n_splits = k, shuffle = True)
    
    
    # save cross validation accuracy/ f1_score
    cv_accuracy=[]
    cv_f1score = []
    
    
    #반복회수 출력을 위해 사용
    n_iter = 0
    

    # KFold 객체의 split()을 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
    for train_index, test_index in kfold.split(X.values):
        # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
        X_train, X_test = X.loc[train_index, :], X.loc[test_index, :]
        y_train, y_test = y[train_index], y[test_index]

        # 학습 및 예측

        model.fit(X_train,y_train)
        pred = model.predict(X_test)
        n_iter += 1

        # 반복 시마다 정확도/f1score 측정
        accuracy = np.round(accuracy_score(y_test, pred), 4) * 100
        f1score = np.round(f1_score(y_test, pred), 4) * 100
        
        train_size = X_train.shape[0]
        test_size = X_test.shape[0]

        print('#{} 검증 정확도 : {:.2f} %   // 검증 f1 score : {:.2f}'.format(n_iter, accuracy, f1score))
        
        cv_accuracy.append(accuracy)
        cv_f1score.append(f1score)

    print('\n평균 정확도:{:.2f} % '.format(np.mean(cv_accuracy)))
    print('평균 f1score:{:.2f} % '.format(np.mean(cv_f1score)))
    display(pd.DataFrame(confusion_matrix(y_test, pred)))
    
    
    # 8번의 교차검증 후, 마지막에 실시한 교차검증의 모델을 return
    return model , X_test, y_test , test_index
    
clf_lgbm, X_test, y_test, test_index = printKfoldAcc(model = lightgbm.LGBMClassifier(), k = 10)

#### model : LGBMClassifier() ####
#1 검증 정확도 : 86.57 %   // 검증 f1 score : 86.32
#2 검증 정확도 : 86.62 %   // 검증 f1 score : 86.66
#3 검증 정확도 : 86.57 %   // 검증 f1 score : 86.42
#4 검증 정확도 : 86.34 %   // 검증 f1 score : 86.05
#5 검증 정확도 : 86.83 %   // 검증 f1 score : 86.68
#6 검증 정확도 : 86.96 %   // 검증 f1 score : 86.66
#7 검증 정확도 : 86.76 %   // 검증 f1 score : 86.35
#8 검증 정확도 : 85.74 %   // 검증 f1 score : 85.08
#9 검증 정확도 : 87.35 %   // 검증 f1 score : 87.20
#10 검증 정확도 : 86.65 %   // 검증 f1 score : 86.67

평균 정확도:86.64 % 
평균 f1score:86.41 % 


,0,1
0,1655,215
1,296,1661


In [12]:
X.columns = list(range(0, 300)) + use_col_list
X

,0,1,2,3,4,5,6,7,8,9,...,299,시각,수정시간,화나요rate,카테고리,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
0,-0.001918,0.007998,0.002489,0.007643,-0.001403,-0.007737,0.005334,-0.002964,0.002120,-0.003124,...,-0.000511,201907.0,0.0,0.833333,0.0,2.0,1.0,0.0,15.0,0.0
1,-0.013315,-0.004529,-0.008958,-0.020693,0.003405,-0.008826,-0.001436,0.008990,0.011692,-0.012936,...,-0.007025,202010.0,0.0,0.086957,0.0,125.0,1.0,0.0,12.0,0.0
2,0.016877,0.035185,0.004884,0.057658,0.015763,0.030377,0.052021,-0.001252,-0.034525,-0.024138,...,0.030889,202006.0,0.0,0.690323,0.0,135.0,3.0,3.0,321.0,3.0
3,-0.006251,-0.005613,0.011302,-0.014256,0.014393,0.004637,-0.019988,0.010955,0.014318,0.008478,...,0.000821,202008.0,0.0,0.909091,2.0,3.0,0.0,0.0,40.0,1.0
4,-0.001605,-0.001677,0.001361,0.000050,0.006276,-0.000179,0.002666,0.000098,0.001460,-0.003061,...,0.002538,202012.0,0.0,0.900000,0.0,1.0,0.0,0.0,18.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38273,0.003389,-0.013936,-0.018267,-0.012623,0.013753,0.007380,0.029916,0.008607,-0.007354,0.024146,...,-0.009506,202012.0,0.0,0.976571,2.0,58.0,7.0,6.0,4335.0,33.0
38274,-0.000136,0.013622,0.030080,-0.022355,0.003092,-0.019142,-0.007795,0.005106,-0.000708,0.001959,...,-0.019859,202012.0,0.0,0.947059,2.0,3.0,11.0,8.0,644.0,14.0
38275,0.001652,-0.013804,-0.007904,-0.004676,-0.001274,-0.005148,0.007411,-0.001050,0.000985,0.006988,...,-0.003893,202012.0,0.0,1.000000,2.0,0.0,0.0,0.0,79.0,0.0
38276,0.002527,0.010039,-0.005158,-0.016506,0.003681,-0.008477,0.023262,0.002881,-0.013556,0.014839,...,-0.004764,202012.0,0.0,0.928571,2.0,2.0,0.0,1.0,52.0,1.0


In [13]:
FIdf = pd.DataFrame([X.columns, clf_lgbm.feature_importances_]).T
FIdf = FIdf.rename(columns = { 0 : 'feature' , 1 : 'importances'})

FIdf['importances'] = FIdf['importances'].astype(int)
FIdf.sort_values('importances', ascending = False).head(30)

,feature,importances
300,시각,200
307,화나요,165
302,화나요rate,130
304,좋아요,126
303,카테고리,67
308,후속기사원해요,55
165,165,31
301,수정시간,30
253,253,27
128,128,26


In [14]:
print(X_test.shape)
X_test_pred_proba = clf_lgbm.predict_proba(X_test)
X_test_pred_proba

(3827, 309)


array([[0.39352687, 0.60647313],
       [0.22094392, 0.77905608],
       [0.80695188, 0.19304812],
       ...,
       [0.91759679, 0.08240321],
       [0.08242555, 0.91757445],
       [0.27380744, 0.72619256]])

In [15]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,299,300,301,302,303,304,305,306,307,308
3,-0.006251,-0.005613,0.011302,-0.014256,0.014393,0.004637,-0.019988,0.010955,0.014318,0.008478,...,0.000821,202008.0,0.0,0.909091,2.0,3.0,0.0,0.0,40.0,1.0
9,-0.006123,0.003220,0.003449,0.003750,0.017432,0.008659,-0.005827,-0.010262,-0.012571,-0.022630,...,0.020136,201910.0,0.0,0.953582,0.0,38.0,2.0,2.0,945.0,4.0
10,0.000535,-0.009354,0.005179,0.003262,0.009643,0.004508,0.005670,-0.007104,0.000556,-0.005476,...,0.010419,202008.0,0.0,0.800000,0.0,4.0,0.0,0.0,16.0,0.0
13,0.008308,0.001938,0.006228,-0.003243,0.021479,0.005213,0.014520,0.000546,-0.015697,0.001373,...,0.001662,201907.0,0.0,0.000000,2.0,17.0,0.0,0.0,0.0,0.0
29,-0.000965,-0.002402,-0.007990,0.001830,0.011787,-0.000810,-0.001289,0.000338,-0.007986,-0.009355,...,0.004275,202006.0,1.0,0.825000,1.0,3.0,0.0,4.0,33.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38231,0.011888,-0.011955,-0.002610,-0.018898,0.011561,-0.002653,0.014628,-0.006275,-0.011983,0.000092,...,0.002204,202012.0,0.0,0.955647,2.0,38.0,12.0,227.0,7649.0,78.0
38252,-0.022230,-0.060247,-0.042317,-0.014771,0.047141,-0.021272,0.002861,0.003077,-0.030882,-0.001299,...,0.037492,202012.0,0.0,0.276786,2.0,80.0,0.0,1.0,31.0,0.0
38259,0.018460,-0.019040,-0.013980,-0.013416,0.019452,0.000264,0.031707,-0.008853,-0.029597,0.010695,...,-0.005765,202012.0,0.0,0.666667,2.0,2.0,0.0,0.0,4.0,0.0
38266,0.007643,-0.003503,0.004829,-0.019410,0.016157,0.004674,0.010442,0.006112,-0.011872,0.006550,...,0.005051,202012.0,0.0,0.958466,2.0,16.0,1.0,1.0,600.0,8.0


In [16]:
finalDF.loc[X_test.index, :]

,시각,카테고리,미디어,제목,내용,수정시간,감정표현,label,화나요rate,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
3,202008,2,뉴시스,조국 기자 뻗치기 취재 저격언론 자유 한계 없나,조국 집 찾아와 취재한 기자들 비판 기자는 질문할 특권 향유하고 있나 딸 집 초인종...,0,"[3, 0, 0, 40, 1]",0,0.909091,3,0,0,40,1
9,201910,0,한국경제TV,文대통령 미래차 청사진 발표‥2030년 세계 1위,한국경제TV 권영훈 기자 앵커 미래자동차라고 하면 자율주행차와 전기차 수소차 등이 ...,0,"[38, 2, 2, 945, 4]",0,0.953582,38,2,2,945,4
10,202008,0,뉴시스,이해찬 국민 성원에 감사앞으로 한반도 평화 위해 노력,유튜브 민주당 채널 통해 퇴임 기자간담회 서울 뉴시스 장세영 기자 이해찬 더불어민주...,0,"[4, 0, 0, 16, 0]",0,0.800000,4,0,0,16,0
13,201907,2,뉴스1,화순 보은병원 철인3종 선수단에 500만원 훈련용품 지원,전남 화순군 도곡면에 자리한 보은병원 병원장 박성록·사진 왼쪽 첫번째 은 8일 광주...,0,"[17, 0, 0, 0, 0]",0,0.000000,17,0,0,0,0
29,202006,1,MBC,삼성 노사관계 자문 그룹 신설이재용 사과 후 후속 조치,삼성이 이재용 부회장의 대국민 사과 이후 노동3권을 실효성 있게 보장하기 위한 구체...,1,"[3, 0, 4, 33, 0]",0,0.825000,3,0,4,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38231,202012,2,조선일보,“일본 유람선보다 더한 코로나 비극 지금 한국 요양병원에”,미소들 요양병원 최희찬 과장 “일본 유람선보다 더한 일들이 요양병원에서 벌어지고 있...,0,"[38, 12, 227, 7649, 78]",1,0.955647,38,12,227,7649,78
38252,202012,2,한국경제,윤석열 국민만 바라보고 좌고우면 하지 않는 국민의 검찰 돼야,윤석열 검찰총장 사진 한경DB 윤석열 검찰총장이 검찰개혁의 목적과 방향은 공정한 검...,0,"[80, 0, 1, 31, 0]",1,0.276786,80,0,1,31,0
38259,202012,2,문화일보,뒷북 법무부 “변호사 접견제한 공정한 재판권리 빼앗는 것”,교정시설 3단계 거리두기 논란 집단감염 2주 만에 대책 내놔 당국 초기대응실패로 재...,0,"[2, 0, 0, 4, 0]",1,0.666667,2,0,0,4,0
38266,202012,2,서울신문,20년전 무고한 청년 옥살이시킨 오심판사 박범계는 사과했다,삼례 나라슈퍼 재심사건 맡았던 박준영 변호사 “불쌍한 청년들에 대한 황당한 오판이었...,0,"[16, 1, 1, 600, 8]",1,0.958466,16,1,1,600,8


In [17]:
testing_df = finalDF.loc[X_test.index, :].reset_index(drop = True).copy()
print('testing_df.shape : ', testing_df.shape)

testing_df.loc[:, 'fake_prob'] = [int(i[1]*100) for i in X_test_pred_proba]
testing_df = testing_df.loc[:,  ['시각','카테고리','미디어','제목','내용','감정표현','fake_prob']]

testing_df

testing_df.shape :  (3827, 14)


,시각,카테고리,미디어,제목,내용,감정표현,fake_prob
0,202008,2,뉴시스,조국 기자 뻗치기 취재 저격언론 자유 한계 없나,조국 집 찾아와 취재한 기자들 비판 기자는 질문할 특권 향유하고 있나 딸 집 초인종...,"[3, 0, 0, 40, 1]",60
1,201910,0,한국경제TV,文대통령 미래차 청사진 발표‥2030년 세계 1위,한국경제TV 권영훈 기자 앵커 미래자동차라고 하면 자율주행차와 전기차 수소차 등이 ...,"[38, 2, 2, 945, 4]",77
2,202008,0,뉴시스,이해찬 국민 성원에 감사앞으로 한반도 평화 위해 노력,유튜브 민주당 채널 통해 퇴임 기자간담회 서울 뉴시스 장세영 기자 이해찬 더불어민주...,"[4, 0, 0, 16, 0]",19
3,201907,2,뉴스1,화순 보은병원 철인3종 선수단에 500만원 훈련용품 지원,전남 화순군 도곡면에 자리한 보은병원 병원장 박성록·사진 왼쪽 첫번째 은 8일 광주...,"[17, 0, 0, 0, 0]",5
4,202006,1,MBC,삼성 노사관계 자문 그룹 신설이재용 사과 후 후속 조치,삼성이 이재용 부회장의 대국민 사과 이후 노동3권을 실효성 있게 보장하기 위한 구체...,"[3, 0, 4, 33, 0]",45
...,...,...,...,...,...,...,...
3822,202012,2,조선일보,“일본 유람선보다 더한 코로나 비극 지금 한국 요양병원에”,미소들 요양병원 최희찬 과장 “일본 유람선보다 더한 일들이 요양병원에서 벌어지고 있...,"[38, 12, 227, 7649, 78]",91
3823,202012,2,한국경제,윤석열 국민만 바라보고 좌고우면 하지 않는 국민의 검찰 돼야,윤석열 검찰총장 사진 한경DB 윤석열 검찰총장이 검찰개혁의 목적과 방향은 공정한 검...,"[80, 0, 1, 31, 0]",58
3824,202012,2,문화일보,뒷북 법무부 “변호사 접견제한 공정한 재판권리 빼앗는 것”,교정시설 3단계 거리두기 논란 집단감염 2주 만에 대책 내놔 당국 초기대응실패로 재...,"[2, 0, 0, 4, 0]",8
3825,202012,2,서울신문,20년전 무고한 청년 옥살이시킨 오심판사 박범계는 사과했다,삼례 나라슈퍼 재심사건 맡았던 박준영 변호사 “불쌍한 청년들에 대한 황당한 오판이었...,"[16, 1, 1, 600, 8]",91


In [18]:
testing_df[testing_df['fake_prob'] >= 80].sample(frac = 0.1).head(5)

,시각,카테고리,미디어,제목,내용,감정표현,fake_prob
2768,202006,0,조선일보,하다하다삐라를 못보내게 군대를 동원하자고 합니다,여권 일제히 대북전단 살포 때리기 정세현 군병력 동원해서 강력히 막아야 김태년 오는...,"[68, 20, 13, 11018, 49]",99
3466,202001,2,매일경제,단독 엘리트 법관 줄사퇴 계속 김정운 서울서부지법 수석부장 사의,판사들 친정부 판사 정치 선언 늑장 사법개혁 편가르기 탓에 법원 점점 뒤숭숭 사진 ...,"[35, 2, 10, 1247, 6]",95
2916,202008,0,이데일리,자기들이 적폐니깐안철수·진중권이 나눈 이야기,이데일리 김민정 기자 안철수 국민의당 대표와 진중권 전 동양대 교수의 대담 영상이 ...,"[371, 3, 2, 22, 2]",83
3337,202007,1,매일경제,집주인들 발동동집팔러 내놨는데 세입자 연락두절,임대차법 시행 혼란 전월세 거주를 4년간 보장하고 전월세 인상률을 5로 제한하는 주...,"[306, 39, 34, 9516, 88]",97
3316,202007,1,조선일보,양도세 2배·취득세 3배·종부세 9배 인상 與 또 세금폭탄 의존,민심 들끓는 부동산 21번의 실패 4 여전히 규제만 쏟아내 문재인 대통령이 지난 2...,"[31, 2, 0, 749, 6]",86


In [19]:
testing_df[testing_df['fake_prob'] <= 10].sample(frac = 0.1).head(5)

,시각,카테고리,미디어,제목,내용,감정표현,fake_prob
504,201906,0,KBS,‘평화 위한 남북 배구스포츠로 교류 ‘물꼬,앵커 앞서 남북 북미관계에 중요한 한 주가 시작된다고 전해드렸는데요 이미 남북이 함...,"[8, 1, 0, 4, 0]",8
420,202002,0,뉴시스,靑 文 내달 中진출 기업에 격려영상재계 건의 전폭 수용종합,총 16개 모든 건의사항 신속한 후속조치 이행 이재용 회장 요청에 내달 文대통령 격...,"[2, 1, 0, 8, 0]",3
477,202005,2,뉴시스,수도권 등교 상황 점검 회의장 입장하는 유은혜 부총리,세종 뉴시스 강종민 기자 고3 학생들의 등교 개학이 시작된 20일 오전 유은혜 부총...,"[0, 0, 1, 36, 2]",10
494,201904,0,뉴스1,김정은푸틴 오늘 첫 대면8년 만에 북러 정상회담,오후 1시께 실무 오찬 가능성김정은 체류 일정에 주목 김정은 북한 국무위원장이 지난...,"[3, 0, 0, 12, 1]",2
77,202007,2,뉴스1,음주단속 경찰 치고 달아난 20대 ‘징역5년 구형,© News1 DB 천안 뉴스1 김아영 기자 음주운전 단속을 하는 경찰을 치고 달아...,"[2, 0, 0, 10, 0]",4


In [20]:
[i[1] for i in X_test_pred_proba]

[0.6064731336904189,
 0.7790560807703126,
 0.193048124762848,
 0.059333651660118024,
 0.4594900368619852,
 0.5599792982169863,
 0.4099422262713633,
 0.7557989230166023,
 0.1870046128821147,
 0.03773206546391203,
 0.9599615424367598,
 0.278140705859785,
 0.2669523933721176,
 0.46369366396178785,
 0.039624201451336945,
 0.30635812104402027,
 0.3879977201519849,
 0.3608990438019616,
 0.6453039801461861,
 0.21523752839712637,
 0.41868455278875705,
 0.008342883695600774,
 0.5608942039889842,
 0.20407215628423975,
 0.03799721958147145,
 0.3032607199643459,
 0.6598761807214749,
 0.029094256131445857,
 0.4867961106084597,
 0.10488119859960787,
 0.8242848725230641,
 0.8986376199565776,
 0.17069910031415586,
 0.03680156496582536,
 0.47598491906887913,
 0.1938943857784206,
 0.012717331834655732,
 0.060274572228273635,
 0.2588845480565127,
 0.5419824902021194,
 0.135384170584797,
 0.15637241189341672,
 0.7059569693395595,
 0.014401800736554869,
 0.009587523114293622,
 0.01158086540361047,
 0.10050

학습하지 않은 test 데이터에 대한 결과

In [21]:
test_pred_fake_index = X_test[np.array([i[1] for i in clf_lgbm.predict_proba(X_test)]) > 0.99].index

finalDF.loc[test_pred_fake_index, :].sample(frac = 0.2).head(6)

,시각,카테고리,미디어,제목,내용,수정시간,감정표현,label,화나요rate,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
28783,202007,0,조선일보,이순신도 관노와 잠자리 도 넘은 박원순 감싸기,진중권 이걸 말이라고친문 눈엔 국민이 노비 박원순은 이순신 아냐 피해여성도 관노 아...,1,"[123, 48, 32, 20493, 132]",1,0.983916,123,48,32,20493,132
29567,202008,0,조선일보,심판자인양 친일판결 나라 잃은 백성들에게 왜 죄 묻는가,김원웅 광복절 기념사에 반박 연설 원희룡 제주지사 인터뷰 원희룡 제주지사는 지난 1...,1,"[9782, 27, 9, 162, 32]",1,0.016181,9782,27,9,162,32
29935,202009,0,중앙일보,제2 조국사태 걱정하는 여당秋 싸우는 바람에 일커졌다,추미애 법무부 장관 연합뉴스 더불어민주당 의원들이 야당의 추미애 법무부 장관 아들에...,1,"[44, 11, 2, 5775, 41]",1,0.983313,44,11,2,5775,41
30223,202009,0,연합뉴스,사오정이냐정경두 秋아들 군 의혹에 마지막 국회서 진땀,실제 치료 일수만큼 휴가가 규정에 맞다 답변했다가 정정 군 출신 신원식 40년 군생...,1,"[16, 6, 2, 2920, 20]",1,0.985155,16,6,2,2920,20
37572,202012,2,서울경제,단독 윤석열 특활비 논란 이성윤 입에서 시작,李지검장 특활비 안줘 하소연 심재철 검찰국장 秋장관에 보고 이성윤 서울중앙지검장 연...,1,"[43, 12, 6, 5056, 20]",1,0.984232,43,12,6,5056,20
32351,202012,0,중앙일보,끝내 실패한 윤석열 찍어내기文 ‘레임덕 차단 둑 무너졌다,윤석열 검찰총장 징계 효력정지 결정이 내려진 다음날인 25일 김남국 왼쪽부터 신동근...,1,"[8950, 148, 12, 99, 41]",1,0.010703,8950,148,12,99,41


In [22]:
finalDF.loc[test_pred_fake_index, :].sample(frac = 0.2)[['시각','카테고리','미디어','제목','내용','수정시간','감정표현','화나요rate']]

,시각,카테고리,미디어,제목,내용,수정시간,감정표현,화나요rate
33348,202007,1,조선일보,1주택자 세금 변화 없다 김현미 발언에 현실 알고 말하나,종부세율 이미 올리고 공시가격 급등시켜 네티즌 온 국민을 적폐세력으로 몰고 있다 발...,1,"[63, 9, 8, 6328, 43]",0.980933
24248,201910,2,서울경제,속보“‘정경심 이름 서류·파일 다 지워라” 조국 5촌 조카 증거인멸 감행,조국 법무부 장관 가족을 둘러싼 사모펀드 투자 의혹의 ‘몸통인 조 장관의 5촌 조카...,1,"[6, 4, 1, 2116, 14]",0.988323
32677,202002,1,한국경제,1인당 최대 4089만원청년층 현금 복지 둑 터졌다,대한민국은 수당천국 2 쏟아지는 청년 현금지원 학비·교통비·주거비·면접수당 정부·지...,1,"[163, 27, 23, 16217, 78]",0.982372
30702,202010,0,이데일리,개천절 드라이브 스루 집회 조건부 허용하자양심적인 판사,이데일리 박지혜 기자 법원이 개천절인 오는 3일 서울 일부 지역에서의 ‘드라이브 스...,1,"[1872, 21, 10, 842, 15]",0.305072
29963,202009,0,중앙일보,YS아들 김현철 아들아 미안하다 엄마가 추미애 아니라서,김현철 김영삼민주센터 상임이사 뉴스1 김현철 김영삼민주센터 상임이사가 아들아 미안하...,1,"[5273, 15, 7, 212, 31]",0.038281
20445,201909,0,조선일보,단독 장학금 서류 폐기했다던 서울대 총동창회의 거짓말,조국 의혹 확산 검찰 압수수색서 당시 서류 확보 조국 법무부 장관 후보자는 지난 2...,1,"[48, 15, 12, 12203, 73]",0.988017
20106,201908,0,중앙일보,단독조국이 펀드에 투자한 후 관급공사 2년간 177건 수주,가로등 점멸기 업체 웰스씨앤티 펀드서 지분 인수 뒤 수주 급증 지자체·공공기관 44...,1,"[29, 8, 10, 9225, 83]",0.986104
20157,201908,0,국민일보,“사회가 싫다는 데 어쩔건가” 조국 논란에 입 연 우석훈 박사,2008년 20대의 95가 비정규직 노동자가 될 것이라는 예측을 담아 화제를 모은 ...,1,"[3676, 7, 10, 90, 14]",0.023703
24641,201911,2,국민일보,단독 조국 “ATM 통해 돈 보냈지만 WFM 주식 매입할 줄 몰랐다”,수천만원 송금 변호인에 첫 인정 저가매수·부당이득엔 무관 주장 사진 연합뉴스 검찰 ...,0,"[22, 6, 7, 4691, 33]",0.985711
31106,202010,0,조선일보,親文지지층 금태섭에 “함께해서 더러웠고 만나지 말자”,금태섭 “소통의 문화를 찾아볼 수 없다” 친여 지지층 “소신같은 소리하고 자빠져” ...,1,"[255, 19, 28, 11621, 61]",0.969710


In [23]:
test_pred_true_index = X_test[np.array([i[0] for i in clf_lgbm.predict_proba(X_test)]) > 0.992].index

finalDF.loc[test_pred_true_index, :].sample(frac = 0.2)

,시각,카테고리,미디어,제목,내용,수정시간,감정표현,label,화나요rate,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
4559,202004,1,파이낸셜뉴스,피겨 퀸 김연아 TS샴푸 모델로 선정,국내 탈모샴푸 시장의 부동의 1위 브랜드 ‘TS샴푸를 제조 판매하는 TS트릴리온 대...,0,"[55, 7, 1, 0, 0]",0,0.000000,55,7,1,0,0
3022,201903,2,뉴스1,단독이재명 친형 2002년 조증약 한 차례 복용,2012년 검찰 공소장서 재선씨 진술 공개 4일 오전 10시 수원지법 성남지원서 7...,1,"[35, 1, 3, 229, 1]",0,0.851301,35,1,3,229,1
18387,201901,2,뉴시스,서울역에 문재인 대통령 생일 축하 광고,서울 뉴시스 배훈식 기자 23일 오후 서울 용산구 서울역 고속철도 2번 출구 옥외 ...,0,"[160, 8, 0, 9, 0]",0,0.050847,160,8,0,9,0
5699,201903,1,아시아경제,LG전자 인도 상업용 에어컨 시장 평정,올해 매출 30 성장 목표 LG전자사진 연합뉴스 아시아경제 국제경제팀 LG전자가 올...,1,"[18, 0, 0, 1, 2]",0,0.047619,18,0,0,1,2
10614,201903,1,파이낸셜뉴스,SK하이닉스 반도체 공급용 자가발전소 건설에 1조6800억 투자,전력 수급 안전성을 위한 공급선 다변화 목적 지난해 11월 이사회 보류 후 3개월만...,0,"[22, 0, 0, 2, 0]",0,0.083333,22,0,0,2,0
6101,201902,2,뉴시스,선거법 위반 혐의 하유정 충북도의원 국민참여재판 불복 인용종합,대전고법 국민참여재판 배제 결정 즉시항고 인용 검찰 재항고 없을 땐 무작위 배심원 ...,0,"[0, 0, 0, 13, 2]",0,0.866667,0,0,0,13,2
18601,201903,2,뉴시스,탈세 혐의 아레나 실소유주 구속심사 기습 출석,오전 10시30분 송경호 판사 심리로 취재진 피해 다른 입구로 법정 입장 서울 뉴시...,1,"[1, 1, 0, 10, 1]",0,0.769231,1,1,0,10,1
3805,201902,2,뉴시스,버닝썬 의혹 애나 강제추행 피해자 조사후 귀가종합,애나 버닝썬 사태 촉발 김모씨 고소장 오후 3시35부터 약 3시간 조사후 귀가 전날...,0,"[1, 0, 0, 43, 2]",0,0.934783,1,0,0,43,2


In [24]:
finalDF.loc[test_pred_fake_index, :].sample(frac = 0.2).head(1).index[0]

25510

In [25]:
ARTICLE_ROW = 800

top5_sim_article_index = [i[0] for i in doc2vec_model.docvecs.most_similar(ARTICLE_ROW, topn = 5)]

display(finalDF.loc[[ARTICLE_ROW], :])
display(finalDF.loc[top5_sim_article_index, :])

print([i[1] for i in doc2vec_model.docvecs.most_similar(ARTICLE_ROW, topn = 5)])

,시각,카테고리,미디어,제목,내용,수정시간,감정표현,label,화나요rate,좋아요,훈훈해요,슬퍼요,화나요,후속기사원해요
800,202001,0,뉴스1,부승찬 전 국방 정책보좌관 총선 제주시 을 출사표,민주당 소속군·국회·정부·학계 경험·네트워크 피력 제2공항 건설에는 필요하지만 절차...,0,"[6, 0, 0, 20, 1]",0,0.740741,6,0,0,20,1


KeyError: "None of [Index(['news_tag1692', 'news_tag2492', 'news_tag35620', 'news_tag922',\n       'news_tag31308'],\n      dtype='object')] are in the [index]"

In [ ]:
stop

In [ ]:
ARTICLE_ROW = 800

top5_sim_article_index = [i[0] for i in doc2vec_model.docvecs.most_similar(ARTICLE_ROW, topn = 50)]

display(finalDF.loc[[ARTICLE_ROW], :])
display(finalDF.loc[(top5_sim_article_index), finalDF['시각'] == finalDF.loc[ARTICLE_ROW, '시각'], :])

print([i[1] for i in doc2vec_model.docvecs.most_similar(ARTICLE_ROW, topn = 50)])

In [ ]:
finalDF.loc[(top5_sim_article_index) & (finalDF['시각'] == finalDF.loc[ARTICLE_ROW, '시각']), :]


In [ ]:
(top5_sim_article_index) & (np.where(finalDF['시각'] == finalDF.loc[ARTICLE_ROW, '시각'])[0])

In [ ]:
np.where(finalDF['시각'] == finalDF.loc[ARTICLE_ROW, '시각'])[0]

###### 여기서부터는 유사한 기사 추출해 내고자 구현한 코드로 삭제가능  

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))



In [ ]:
date_value = finalDF['시각'].values
date_value = (date_value - np.mean(date_value) ) / np.std(date_value)

In [ ]:
text_date_vector = np.hstack((X_text, date_value.reshape((-1, 1))))
print(text_date_vector.shape)

In [ ]:
cos_sim(text_date_vector[0], text_date_vector[1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%%time

a = cosine_similarity(text_date_vector)
a

In [ ]:
text_date_vector

In [ ]:
doc2vec_model.docvecs.

In [ ]:
# import numpy as np
# from scipy import sparse
# from sklearn.metrics.pairwise import cosine_similarity

# sparse_mat = sparse.csr_matrix(text_date_vector)

# similarities = cosine_similarity(sparse_mat)

In [ ]:
mat = np.random.rand(65000, 10)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_n_space(m1, m2, batch_size=100):
    assert m1.shape[1] == m2.shape[1]
    ret = np.ndarray((m1.shape[0], m2.shape[0]))
    for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
        start = row_i * batch_size
        end = min([(row_i + 1) * batch_size, m1.shape[0]])
        if end <= start:
            break # cause I'm too lazy to elegantly handle edge cases
        rows = m1[start: end]
        sim = cosine_similarity(rows, m2) # rows is O(1) size
        ret[start: end] = sim
    return ret

속도가 안됨..

In [ ]:
text_date_vector.astype(np.float32)

In [ ]:
from scipy import sparse

sparse.csr_matrix(text_date_vector).shape

In [ ]:
from sklearn.decomposition import TruncatedSVD


svd = TruncatedSVD(n_components= 20)

U = svd.fit_transform(text_date_vector)

Sigma = svd.explained_variance_ratio_

VT = svd.components_

svd.fit(text_date_vector)

In [ ]:
U.shape

In [ ]:
%%time
cosine_similarity(sparse.csr_matrix(U))

##### 삭제 ~

In [ ]:
finalDF